#### 1. 불러오기

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('./train.csv')
train_x = train.drop(['ID', 'vehicle_speed(km/h)'], axis=1)
train_y = train['vehicle_speed(km/h)']

test_x = pd.read_csv('./test.csv')
test_x = test_x.drop(['ID'], axis = 1)

In [ ]:
train.head()

In [ ]:
train_x.head()

In [ ]:
train_x = train_x.drop(columns=['brake_cmd'], axis=1)
test_x = test_x.drop(columns=['brake_cmd'], axis=1)

#### 3. 데이터 전처리

컴퓨터는 숫자만 계산 가능하기 때문에 문자형으로 이루어진 범주형 변수들을 분석에 사용하려면 숫자형으로 변환해 주어야 합니다. <br>
본 데이터셋에는 문자형 변수가 이미 숫자형으로 표현되어 있기 때문에, 이를 모델 학습에 보다 적절한 형태로 가공하는 방법을 살펴보겠습니다.

In [ ]:
train_x.head()

#### 3-1. 스케일링

데이터 모델링에서 스케일링은 다차원의 값들을 비교 분석하기 쉽게 만들어 자료의 오버플로우(overflow)나 언더플로우(underflow)를 방지하고 최적화 과정에서의 안정성 및 수렴속도를 향상시킬 수 있습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#각 변수들의 분포를 고려하여 스케일링을 진행합니다.
#분포가 중앙으로 몰려있는 경우, 최솟값을 0, 최댓값을 1로 변환하는 MinMaxScaler 적용
#분포가 한 쪽으로 치우친 형태의 경우, 평균과 표준편차를 이용한 StandardScaler 적용
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

minmax_columns = ['X1', 'X2', 'steering_angle(deg)', 'longitudinal_acceleration(m/s^2)', 'lateral_acceleration(m/s^2)', 'UTM_x', 'UTM_y']
standard_columns = ['acceleration_cmd']

scaled_train_x = train_x.copy()
scaled_train_x[minmax_columns] = minmax_scaler.fit_transform(train_x[minmax_columns])
scaled_train_x[standard_columns] = standard_scaler.fit_transform(train_x[standard_columns])

scaled_test_x = test_x.copy()
# [주의] 단, 테스트데이터에는 transform만 적용
scaled_test_x[minmax_columns] = minmax_scaler.transform(test_x[minmax_columns])
scaled_test_x[standard_columns] = standard_scaler.transform(test_x[standard_columns])

#### 4. 분석 모델 설계

선형 모델인 Linear Regression, 트리 기반 모델 중 배깅 기법을 적용한 Random Forest, ExtrTrees, 트리 기반 모델 중 부스팅 기법을 적용한 XGBoost와 Light GBM을 사용하여 어떤 모델이 좋은 성능을 보여주는지 실험합니다.

In [ ]:
lr = LinearRegression()
rf = RandomForestRegressor()
et = ExtraTreesRegressor()
xgb = XGBRegressor()
lgbm = LGBMRegressor()

#### 4-1. 분석 모델 성능 평가

성능을 평가하기 위해 학습 데이터셋에서 20%를 활용해 검증 데이터셋을 만듭니다. <br>
이때, 분할된 학습데이터를 사용하여 최종 모델을 학습시키는 것은 검증 데이터셋으로 할당한 20%만큼의 데이터 손실이 발생한다는 점을 유의하시기 바랍니다.

In [ ]:
train_x_splited, val_x, train_y_splited, val_y = train_test_split(scaled_train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
lr.fit(train_x_splited, train_y_splited)
lr_pred = lr.predict(val_x)
print('Linear Regression 모델의 RMSE : ', np.sqrt(mean_squared_error(val_y, lr_pred)))

rf.fit(train_x_splited, train_y_splited)
rf_pred = rf.predict(val_x)
print('Random Forest 모델의 RMSE : ', np.sqrt(mean_squared_error(val_y, rf_pred)))

et.fit(train_x_splited, train_y_splited)
et_pred = et.predict(val_x)
print('Extra Trees 모델의 RMSE : ', np.sqrt(mean_squared_error(val_y, et_pred)))

xgb.fit(train_x_splited, train_y_splited)
xgb_pred = xgb.predict(val_x)
print('XGBoost 모델의 RMSE : ', np.sqrt(mean_squared_error(val_y, xgb_pred)))

lgbm.fit(train_x_splited, train_y_splited)
lgbm_pred = lgbm.predict(val_x)
print('LGBM 모델의 RMSE : ', np.sqrt(mean_squared_error(val_y, lgbm_pred)))

#### 4-2. GridSearchCV(Gridsearch Cross Validation)를 활용한 하이퍼파라미터 튜닝

이제 검증 데이터셋을 이용한 평가에서 가장 좋은 성능을 보인 Extra Trees, LGBM 모델의 하이퍼 파라미터 튜닝을 진행하기 위해 Gridsearch CV를 활용합니다.<br>
일반적으로 모델의 성능에 가장 직접적인 영향을 주는 파라미터는 n_estimators이므로, 이 값을 100에서 2000 사이로 변경해보면서 모델의 성능을 측정해 봅니다.<br>
이때 파라미터 cv는 5의 값을 갖는데, 주어진 데이터셋을 몇 개의 fold로 나눌지를 결정합니다.

In [ ]:
param_grid = [
    {'n_estimators' : [100, 500, 1000, 2000],}
]

et = ExtraTreesRegressor(random_state=42)

grid_search_et = GridSearchCV(et,
                           param_grid,
                           cv=5,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)

grid_search_et.fit(scaled_train_x, train_y)

print('최적의 하이퍼파라미터 : ', grid_search_et.best_params_)

이번에는 모델 성능 평가에서 두 번재로 좋은 결과를 보였던 Random Forest Model에 대해서도 하이퍼 파라미터 튜닝을 진행합니다.

In [ ]:
param_grid = [
    {'n_estimators' : [100, 500, 1000, 2000],}
]

rf = RandomForestRegressor(random_state=42)

grid_search_rf = GridSearchCV(rf,
                           param_grid,
                           cv=5,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)

grid_search_rf.fit(scaled_train_x, train_y)

print('최적의 하이퍼파라미터 : ', grid_search_rf.best_params_)

#### 5. 모델 학습

In [ ]:
et = ExtraTreesRegressor(**grid_search_et.best_params_, random_state=42)
et.fit(scaled_train_x, train_y)

In [ ]:
rf = RandomForestRegressor(**grid_search_rf.best_params_, random_state=42)
rf.fit(scaled_train_x, train_y)

#### 6. 예측값 생성

In [ ]:
et_pred = et.predict(scaled_test_x)

In [ ]:
rf_pred = rf.predict(scaled_test_x)

#### 6-1. 앙상블

앙상블은 여러 모델로부터의 얻은 예측을 결합합으로써 보다 정확한 예측을 도출하는 기법입니다. <br>
앙상블 기법 중 하나인 하드보팅은 모델의 예측 결과를 더하여 구한 값을 모델의 개수로 나눠 평균을 취하는 방법을 뜻합니다.

In [ ]:
mean = (et_pred + rf_pred) / 2

#### 7. 제출 파일 생성

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.head(5)

In [ ]:
submission['vehicle_speed(km/h)'] = mean
submission.head(5)

In [ ]:
submission.to_csv('./ensemble_submit.csv', index = False)